In [1]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import random


In [2]:
#!conda install -y pip
#!conda install -y -c conda-forge selenium=4.16
#!pip install webdriver_manager

In [3]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'/home/diegogl/.wdm/drivers/geckodriver/linux64/v0.34.0/geckodriver'

In [4]:
#Instalar gecko
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service


#Abrir un navegador
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Firefox()

In [5]:
URL="https://cpu.userbenchmark.com/"

In [6]:

def waitFinishLoad(driver,idLoader,needsDisapeared=False,by=By.ID,needsWait=True):
    """Espera a la carga de un elemento antes de continuar

    Args:
        driver (WebDriver): WebDriver
        idLoader (str): Id a buscar, puede ser XPath...
        needsDisapeared (bool, optional): Si el elemento a esperar tiene que aparecer o desaparecer como un CircularProgress. Defaults to False.
        by (str, optional): Tipo de By a buscar el Wait. Defaults to By.ID.
        needsWait (bool, optional): Si necesita esperar un extra para que no pueda saltar un Captcha. Defaults to False.
    """
    if idLoader is None:
        return
    WebDriverWait(driver, timeout=60).until(EC.presence_of_element_located((by, idLoader)))
    if needsDisapeared:
        WebDriverWait(driver, timeout=60).until(EC.invisibility_of_element((by, idLoader)))
    if needsWait:
        time.sleep(random.choice([0.05,0.1,0.2]))

def elementBy(query,driver,**kargs):
    """Devuelve un element|None dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        Element|None: Elemento de la búsqueda
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_element(kargs.get("by",By.XPATH),query)
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element

def clickEvent(driver,element,**kargs):
    """Click en un elemento que funciona con evento

    Args:
        driver (WebDriver): WebDriver
        element (Element): Elemento que tiene el evento
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    if kargs.get("normal",False):
        element.click
    else:
        try:
            driver.execute_script(element.get_dom_attribute("onClick"))
        except:
            element.click
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))

# endregion

In [7]:
driver.get(URL)

In [8]:
elementBy("//*[contains(text(),'I understand')]",driver,idLoader="//*[contains(text(),'I understand')]",by=By.XPATH,needsWait=True).click()

NoSuchElementException: Message: Unable to locate element: //*[contains(text(),'I understand')]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [57]:
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup
import re

column_names = ["Name","User rating %","Value %","Avg. Bench %", "Memory Pts", "8-Core Pts","Mkt. share %","Age months", "Price €"]
dfT=pd.DataFrame([],columns=column_names)
while True:
    try:
        nextB=elementBy("//*[contains(text(),'Next »')]",driver,idLoader="//*[contains(text(),'Next »')]",by=By.XPATH,needsWait=True)
    except:
        break
    soup=BeautifulSoup(driver.page_source,'html.parser')
    table=soup.findAll("table")[1]
    # Eliminar los números de ranking
    rows = table.find_all('tr')
    rowsL=[]
    for row in rows:
        cells = row.find_all('td')
        cellValues=[]
        for index,cell in enumerate(cells):
            if index == 0:
                continue
            if index==1:
                cellValues.append(cell.find_all("span")[0].text.replace("Compare","").split("\t")[-1].replace("\n",""))
            else:
                try:
                    if index == 4:
                        cellValues.append(cell.find_all("div")[0].text.split("\t")[-1].replace("\n","").replace(cell.find_all("div")[0].find("div").find("div").text,""))
                        continue
                    if len(cell.find_all("div"))==0:
                        cellValues.append(0)
                    cellValues.append(cell.find_all("div")[0].text.split("\t")[-1].replace("\n",""))
                except:
                    None
        rowsL.append(cellValues)

    df = pd.DataFrame(columns=column_names,data=rowsL)
    dfT=pd.concat([dfT,df])
    page=elementBy("//*[contains(text(),'Page ')]",driver,idLoader="//*[contains(text(),'Page ')]",by=By.XPATH,needsWait=True)
    nextB=elementBy("//*[contains(text(),'Next »')]",driver,idLoader="//*[contains(text(),'Next »')]",by=By.XPATH,needsWait=True)
    nextB.click()
    waitFinishLoad(driver,idLoader=f"//*[contains(text(),'Page {int(page.text.split(' ')[1])+1}')]",by=By.XPATH)


In [58]:
dfT['none_count'] = df.apply(lambda x:x.isnull().sum(), axis=1)

# Elimina las filas que tienen más de 3 valores None
dfT = dfT[dfT['none_count'] <= 3]

dfT.drop(columns=["none_count"],inplace=True)

dfT

,Name,User rating %,Value %,Avg. Bench %,Memory Pts,8-Core Pts,Mkt. share %,Age months,Price €
1,Intel Core i5-13600K,▲103▼,112,122,83.9,1454,0.59,17,€319
2,Intel Core i5-12600K,▲98▼,113,109,80,1223,0.83,29,€210
3,Intel Core i5-12400F,▲98▼,114,98,81.6,974,1.77,27,€136
4,Intel Core i3-12100F,▲98▼,121,94.1,83.8,788,0.41,27,€96
5,Intel Core i7-13700K,▲96▼,109,126,84.7,1676,1.09,17,€404
...,...,...,...,...,...,...,...,...,...
46,Intel Core i7 975,▲50▼,0.0,72.4,91.7,408,0,124+,0.0
47,Intel Core2 Duo E4400,▲50▼,0.0,33.3,60.1,41.3,0,122+,0.0
48,Intel Celeron E3200,▲50▼,0.0,35.7,61.7,60.5,0,120+,0.0
49,Intel Core2 4400,▲50▼,0.0,33.1,59.7,40.9,0,121+,0.0


In [59]:
import os
import pathlib

#for i in dfT.columns:
#    dfT[i] = dfT[i].str.replace("th","")
#    dfT[i] = dfT[i].str.replace("nd","")
#    dfT[i] = dfT[i].str.replace("rd","")
#    dfT[i] = dfT[i].str.replace("st","")
dfT['Name'] = dfT['Name'].str.replace("Compare  ","").str.split("Samples").str[0]
#dfT['Avg. bench  %'] = dfT['Avg. bench  %'].str[:-3]
dfT['User rating %'] = dfT['User rating %'].str.replace("▼","").str.replace("▲","")

In [60]:
dfL=dfT.copy(deep=True)

In [61]:
dfT=dfL
dfT

,Name,User rating %,Value %,Avg. Bench %,Memory Pts,8-Core Pts,Mkt. share %,Age months,Price €
1,Intel Core i5-13600K,103,112,122,83.9,1454,0.59,17,€319
2,Intel Core i5-12600K,98,113,109,80,1223,0.83,29,€210
3,Intel Core i5-12400F,98,114,98,81.6,974,1.77,27,€136
4,Intel Core i3-12100F,98,121,94.1,83.8,788,0.41,27,€96
5,Intel Core i7-13700K,96,109,126,84.7,1676,1.09,17,€404
...,...,...,...,...,...,...,...,...,...
46,Intel Core i7 975,50,0.0,72.4,91.7,408,0,124+,0.0
47,Intel Core2 Duo E4400,50,0.0,33.3,60.1,41.3,0,122+,0.0
48,Intel Celeron E3200,50,0.0,35.7,61.7,60.5,0,120+,0.0
49,Intel Core2 4400,50,0.0,33.1,59.7,40.9,0,121+,0.0


In [62]:
from datetime import datetime, timedelta

import pandas as pd

# Supongamos que tu dataframe se llama df y la columna con los meses se llama 'meses'

dfT['Age months'] = dfT['Age months'].replace('\+$', '', regex=True)
# Convertir la columna 'months' a numérico
dfT['Age months'] = pd.to_numeric(dfT['Age months'])

dfT['Age months'] = dfT['Age months'].fillna(0)

# Fecha de inicio es la fecha actual
fecha_inicio = datetime.now()

# Calcula la fecha del evento basado en 'Age months'
dfT['fecha'] = dfT['Age months'].apply(lambda x: fecha_inicio - timedelta(days=int(x)*30))

# Luego, determinamos el cuartil del año
dfT['cuartil'] = pd.cut(dfT['fecha'].dt.dayofyear, bins=[0, 91, 182, 273, 365], labels=['Q1', 'Q2', 'Q3', 'Q4'], include_lowest=True)

# Calcula el año del evento
dfT['año'] = dfT['fecha'].dt.year

# Finalmente, creamos la columna 'Q_año' combinando 'cuartil' y 'año'
dfT['Q_año'] = dfT['cuartil'].astype(str) + ' ' + dfT['año'].astype(str)

In [63]:
dfT["Price €"]=dfT["Price €"].str.replace("€","")

In [64]:
dfT

,Name,User rating %,Value %,Avg. Bench %,Memory Pts,8-Core Pts,Mkt. share %,Age months,Price €,fecha,cuartil,año,Q_año
1,Intel Core i5-13600K,103,112,122,83.9,1454,0.59,17,319,2022-11-21 18:36:56.466897,Q4,2022,Q4 2022
2,Intel Core i5-12600K,98,113,109,80,1223,0.83,29,210,2021-11-26 18:36:56.466897,Q4,2021,Q4 2021
3,Intel Core i5-12400F,98,114,98,81.6,974,1.77,27,136,2022-01-25 18:36:56.466897,Q1,2022,Q1 2022
4,Intel Core i3-12100F,98,121,94.1,83.8,788,0.41,27,96,2022-01-25 18:36:56.466897,Q1,2022,Q1 2022
5,Intel Core i7-13700K,96,109,126,84.7,1676,1.09,17,404,2022-11-21 18:36:56.466897,Q4,2022,Q4 2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,Intel Core i7 975,50,0.0,72.4,91.7,408,0,124,NaN,2014-02-06 18:36:56.466897,Q1,2014,Q1 2014
47,Intel Core2 Duo E4400,50,0.0,33.3,60.1,41.3,0,122,NaN,2014-04-07 18:36:56.466897,Q2,2014,Q2 2014
48,Intel Celeron E3200,50,0.0,35.7,61.7,60.5,0,120,NaN,2014-06-06 18:36:56.466897,Q2,2014,Q2 2014
49,Intel Core2 4400,50,0.0,33.1,59.7,40.9,0,121,NaN,2014-05-07 18:36:56.466897,Q2,2014,Q2 2014


In [65]:
dfT.to_csv(os.path.join(str(pathlib.Path().resolve()),"CPU.csv"))

In [56]:
dfT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1400 entries, 1 to 50
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Name           1400 non-null   object  
 1   User rating %  1400 non-null   object  
 2   Value %        1400 non-null   object  
 3   Avg. Bench %   1400 non-null   object  
 4   Memory Pts     1400 non-null   object  
 5   8-Core Pts     1400 non-null   object  
 6   Mkt. share %   1400 non-null   object  
 7   Age months     1400 non-null   int64   
 8   Price €        157 non-null    object  
 9   año            1400 non-null   int64   
 10  cuartil        1258 non-null   category
 11  Q_año          1400 non-null   object  
dtypes: category(1), int64(2), object(9)
memory usage: 132.8+ KB
